In [16]:
import numpy as np
import copy
import warnings
from qiskit import (
    Aer,
    QuantumCircuit,
    QuantumRegister,
    ClassicalRegister,
    execute,
    transpile
)
from qiskit.utils.mitigation.fitters import CompleteMeasFitter
from qiskit.ignis.verification.tomography import StateTomographyFitter, state_tomography_circuits
from qiskit.quantum_info import state_fidelity
from scipy.linalg import expm
from numpy.linalg import multi_dot 
from numpy import kron as kr
from sympy import Matrix
import os

In [17]:
X = np.array([[0,1],[1,0]]) 
Y = np.array([[0,-1j],[1j,0]])
Z = np.array([[1,0],[0,-1]])
Id = np.eye(2)

# Defining the hamiltonian divided in: 
#       - H1: first two qubits interactions.
#       - H2: second two qubits interactions.
H1 = kr(X, kr(X, kr(Id, Id))) + kr(Y, kr(Y, kr(Id, Id))) + kr(Z, kr(Z, kr(Id, Id))) 
H2 = kr(Id, kr(X, kr(X, Id))) + kr(Id, kr(Y, kr(Y, Id))) + kr(Id, kr(Z, kr(Z, Id)))
H3 = kr(Id, kr(Id, kr(X, X))) + kr(Id, kr(Id, kr(Y, Y))) + kr(Id, kr(Id, kr(Z, Z))) 

In [18]:
Matrix(H1)

Matrix([
[1.0,   0,   0,   0,    0,    0,    0,    0,    0,    0,    0,    0,   0,   0,   0,   0],
[  0, 1.0,   0,   0,    0,    0,    0,    0,    0,    0,    0,    0,   0,   0,   0,   0],
[  0,   0, 1.0,   0,    0,    0,    0,    0,    0,    0,    0,    0,   0,   0,   0,   0],
[  0,   0,   0, 1.0,    0,    0,    0,    0,    0,    0,    0,    0,   0,   0,   0,   0],
[  0,   0,   0,   0, -1.0,    0,    0,    0,  2.0,    0,    0,    0,   0,   0,   0,   0],
[  0,   0,   0,   0,    0, -1.0,    0,    0,    0,  2.0,    0,    0,   0,   0,   0,   0],
[  0,   0,   0,   0,    0,    0, -1.0,    0,    0,    0,  2.0,    0,   0,   0,   0,   0],
[  0,   0,   0,   0,    0,    0,    0, -1.0,    0,    0,    0,  2.0,   0,   0,   0,   0],
[  0,   0,   0,   0,  2.0,    0,    0,    0, -1.0,    0,    0,    0,   0,   0,   0,   0],
[  0,   0,   0,   0,    0,  2.0,    0,    0,    0, -1.0,    0,    0,   0,   0,   0,   0],
[  0,   0,   0,   0,    0,    0,  2.0,    0,    0,    0, -1.0,    0,   0,   0,   0,   0],
[

In [20]:
time = np.pi
n_steps = 10

trotter_step = multi_dot([expm(-time/n_steps*H1*1j),expm(-time/n_steps*H2*1j),expm(-time/n_steps*H3*1j)])

Matrix(trotter_step)

Matrix([
[0.587785252292473 - 0.809016994374947*I,                                        0,                                        0,                                        0,                                       0,                                        0,                                        0,                                       0,                                       0,                                        0,                                        0,                                       0,                                        0,                                        0,                                        0,                                       0],
[                                      0,               0.769420884293814 - 0.25*I,  -0.18163563200134 - 0.559016994374947*I,                                        0,                                       0,                                        0,                                        0,                               